In [13]:
from __future__ import print_function

import pickle
from datetime import datetime
import glob

from pandas import DataFrame, Series
import pandas as pd
import numpy as np
import simplejson
import dateutil.parser

In [14]:
def load_germanwings():
    filename = "../data/germanwings.pkl"
    with open(filename, "rb") as f:
        return pickle.load(f)

german_wings = load_germanwings()

In [15]:
def loader(filespec):
    for filename in glob.glob(filespec):
        print(filename)
        with open(filename) as f:
            for line in f:
                d = simplejson.loads(line)
                doc = {
                    "short_url": d["g"],
                    "country": d.get("c", ""),
                    "timestamp": datetime.utcfromtimestamp(d["t"]),
                    "timezone": d.get("tz", ""),
                }
                if doc["short_url"] in german_wings:
                    yield doc

In [16]:
from csv import DictReader, register_dialect

def load_remapped_timezones():
    print("Loading timezones")
    with open("../data/timezone-map.csv") as f:
        reader = DictReader(f, fieldnames=["timezone", "offset1", "offset2"])
        return {row["timezone"]: row["offset1"] for row in reader}


def load_remapped_country_codes():
    print("Loading country codes")
    dialect = register_dialect('tabs', delimiter='\t')
    with open("../data/country-code-lookup.csv") as f:
        reader = DictReader(f, dialect='tabs')
        return {row["Code"]: row["Country name"] for row in reader}


remapped_timezone = load_remapped_timezones()
remapped_country_code = load_remapped_country_codes()


Loading timezones
Loading country codes


In [17]:
GERMANWINGS_PICKLE = "../data/germanwings-hist.pkl"


def save_germanwings():
    print("Loading log files")
    df = DataFrame(loader("../data-capstone/*.log"))
    print("Adding columns to DataFrame")
    # http://stackoverflow.com/questions/24216425/adding-a-new-pandas-column-with-mapped-value-from-a-dictionary
    df["timezone_offset"] = df.timezone.map(remapped_timezone.get)
    df["country_name"] = df.country.map(remapped_country_code.get)
    # http://stackoverflow.com/questions/25146121/extracting-just-month-and-year-from-pandas-datetime-column-python
    df["day"] = df.timestamp.dt.day
    df["hour"] = df.timestamp.dt.hour
    df["minute"] = df.timestamp.dt.minute
    print("Saving data to pickle file")
    df.to_pickle(GERMANWINGS_PICKLE)
    print("Done")
    return df

def load_germanwings():
    return pd.read_pickle(GERMANWINGS_PICKLE)

In [24]:
df = load_germanwings()
# df = save_germanwings()

In [25]:
def save_as_loadable_JSON(df, json_filename, column_remapping):
    with open(json_filename, "w") as f:
        df_tmp = df.rename(columns=column_remapping)
        objs = [dict(row) for _, row in df_tmp.iterrows()]
        f.write(simplejson.dumps(objs))

In [26]:
TIME_COLUMNS = ["day", "hour", "minute"]
COUNTRY_DF_COLUMNS = ["country_name"] + TIME_COLUMNS
TIMEZONE_COLUMNS = ["timezone_offset"] + TIME_COLUMNS


def country_selection_df(df, countries):
    tmp = df[df.country.isin(countries)]
    s = pd.Series(tmp.groupby(COUNTRY_DF_COLUMNS).count().timezone, name='count')
    return DataFrame(s).reset_index()

def make_US_CA(df):
    countries = ["US", "CA"]
    return country_selection_df(df, countries)

df_US_CA = make_US_CA(df)
save_as_loadable_JSON(df_US_CA, "urlhist_US_CA.json", {'country_name': 'key'})

In [27]:
def make_US_DE_ES_FR_IT(df):
    countries = ["US", "DE", "ES", "IT", "FR"]
    return country_selection_df(df, countries)

df_US_DE_ES_FR_IT = make_US_DE_ES_FR_IT(df)
save_as_loadable_JSON(df_US_DE_ES_FR_IT, "urlhist_US_DE_ES_FR_IT.json", {'country_name': 'key'})

In [28]:
def make_timezone_offset(df):
    s = pd.Series(df.groupby(TIMEZONE_COLUMNS).count().timezone, name='count')
    df_timezone_offset = DataFrame(s)
    df_timezone_offset.reset_index(inplace=True)
    return df_timezone_offset

df_timezone = make_timezone_offset(df)
save_as_loadable_JSON(df_timezone, "urlhist_timezone_offset.json", {'timezone_offset': 'key'})

In [71]:
def short_url_aggregator(df, date_range=None, countries=None, timezones=None):
    tmp = df.copy()
    if date_range:
        start, end = tuple(pd.to_datetime(item) for item in date_range)
        tmp = tmp.query('@start <= timestamp < @end')
    if countries:
        tmp = tmp[tmp.country.isin(countries)]
    if timezones:
        tmp = tmp[tmp.timezone.isin(timezones)]
    s = pd.Series(tmp.groupby("short_url").count().timezone, name='count')
    return DataFrame(s).reset_index()

In [72]:
x = short_url_aggregator(df, date_range=["2015-03-25", "2015-03-26"], countries=["CA"])

In [76]:
d25, d26, d27, d28 = tuple("2015-03-2{0}".format(x) for x in (5, 6, 7, 8))

for country in ["US", "CA", "DE", "IT", "ES", "FR"]:
    for dates in [[d25, d26], [d26, d27], [d27, d28]]:
        print("{0} - {1}".format(country, dates[0]))
        filename = "short_url_count-{0}-{1}.json".format(country, dates[0])
        x = short_url_aggregator(df, countries=[country], date_range=dates)
        filename = "short_url_count-{0}-{1}.json".format(country, dates[0])
        with open(filename, "w") as f:
            objs = [dict(row) for _, row in x.iterrows()]
            objs = {d["short_url"]: d["count"] for d in objs}
            f.write(simplejson.dumps(objs))

US - 2015-03-25
US - 2015-03-26
US - 2015-03-27
CA - 2015-03-25
CA - 2015-03-26
CA - 2015-03-27
DE - 2015-03-25
DE - 2015-03-26
DE - 2015-03-27
IT - 2015-03-25
IT - 2015-03-26
IT - 2015-03-27
ES - 2015-03-25
ES - 2015-03-26
ES - 2015-03-27
FR - 2015-03-25
FR - 2015-03-26
FR - 2015-03-27
